In [2]:
import os
import pandas as pd

# Корневая папка с датасетом (текущая папка, где будет ноутбук)
dataset_dir = os.getcwd()

# Список для хранения данных
data = []

# Проходим по всем подпапкам
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".wav"):
            wav_path = os.path.join(root, file)
            txt_file = file.replace(".wav", ".txt")
            txt_path = os.path.join(root, txt_file)

            # Проверяем, существует ли текстовый файл
            if os.path.exists(txt_path):
                with open(txt_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                
                # Формируем путь относительно Dataset и с прямыми слешами
                rel_path = os.path.relpath(wav_path, dataset_dir).replace("\\", "/")
                
                data.append({
                    "path": rel_path,
                    "text": text
                })

# Создаем DataFrame и сохраняем в CSV
df = pd.DataFrame(data)
csv_path = os.path.join(dataset_dir, "dataset.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"CSV файл создан: {csv_path}")


KeyboardInterrupt: 

In [5]:
import os
import pandas as pd

csv_path = "dataset.csv"

# Загружаем CSV
df = pd.read_csv(csv_path)

df["path"] = "Dataset/" + df["path"]

df = pd.DataFrame(df)
csv_path = os.path.join(dataset_dir, "full_dataset.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

In [10]:
!pip install pandas
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("full_dataset.csv")

train_df, temp_df = train_test_split(
    df,
    test_size=0.2,       # 20% уйдёт на val+test
    random_state=42,
    shuffle=True
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,       # половина temp → test, половина → val
    random_state=42,
    shuffle=True
)

print("Train:", len(train_df))
print("Validation:", len(val_df))
print("Test:", len(test_df))

# 4️⃣ При желании сохраняем в CSV
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

Train: 22488
Validation: 2811
Test: 2812
